In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from build_pandas import build_pandas
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import pandas as pd
import utils

In [9]:
location = "A"
model = "gb"
version = 4
loss = "absolute_error"

In [10]:
x_train, x_test = utils.preprocess_category(location)
x_train.drop(["time", "date_calc"], axis=1, inplace=True)
x_test.drop(["date_calc"], axis=1, inplace=True)
x_train['date_forecast'] = (pd.to_datetime(x_train['date_forecast'], format='%Y') - pd.to_datetime('2000', format='%Y')).dt.total_seconds()
x_test['date_forecast_dt'] = x_test['date_forecast']
x_test['date_forecast'] = (pd.to_datetime(x_test['date_forecast'], format='%Y') - pd.to_datetime('2000', format='%Y')).dt.total_seconds()

print(x_train.columns)
print(x_train.shape)
x_train.head()

Total data points: 34085
Data points to be removed: 0


C:\Code\NTNU\Semester9\ml_prosjekt\utils.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_interpolated.fillna(method='ffill', inplace=True)
C:\Code\NTNU\Semester9\ml_prosjekt\utils.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[us], please explicitly cast to a compatible dtype first.
  merged_df.fillna(0, inplace=True)


Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'date_forecast', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'hour_0', 'hour_1', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_2', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'hour_9', 'is_day:idx', 'is_in_shadow:idx', 'month_1',
       'month_10', 'month_11', 'month_12', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       '

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,date_forecast,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,7.7,1.230,1744.900024,0.0,0.0,1744.900024,612828000.0,0.0,280.299988,0.0,...,342.834015,-3.202,0.0,285.899994,100.0,39640.101562,3.7,-3.6,-0.8,-0.0
1,7.7,1.229,1734.000000,0.0,0.0,1734.000000,612828900.0,0.0,280.299988,0.0,...,346.294006,-3.650,0.0,286.100006,100.0,40123.898438,3.6,-3.6,-0.6,-0.0
2,7.7,1.228,1723.500000,0.0,0.0,1723.500000,612829800.0,0.0,280.299988,0.0,...,349.768005,-3.998,0.0,286.299988,100.0,40628.300781,3.6,-3.6,-0.4,-0.0
3,7.7,1.226,1713.400024,0.0,0.0,1713.400024,612830700.0,0.0,280.299988,0.0,...,353.251007,-4.247,0.0,286.600006,100.0,41153.601562,3.5,-3.5,-0.2,-0.0
4,7.7,1.225,1703.599976,0.0,0.0,1703.599976,612831600.0,0.0,280.299988,0.0,...,356.742004,-4.393,0.0,286.799988,100.0,41699.898438,3.5,-3.5,0.0,-0.0


In [11]:
print(x_test.columns)
x_test.head()

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'date_forecast', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'hour_0', 'hour_1', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_2', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'hour_9', 'is_day:idx', 'is_in_shadow:idx', 'month_1',
       'month_10', 'month_11', 'month_12', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       '

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,date_forecast,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_forecast_dt
0,4.4,1.286,912.700012,0.0,0.0,1041.199951,736214400.0,0.0,271.700012,0.0,...,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0,2023-05-01 00:00:00
1,4.3,1.287,912.700012,0.0,0.0,1054.800049,736215300.0,0.0,271.700012,0.0,...,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0,2023-05-01 00:15:00
2,4.3,1.287,912.700012,0.0,0.0,1068.300049,736216200.0,0.0,271.600006,0.0,...,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0,2023-05-01 00:30:00
3,4.3,1.287,912.700012,0.0,0.0,1081.900024,736217100.0,0.0,271.600006,0.0,...,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0,2023-05-01 00:45:00
4,4.3,1.287,NaN,0.0,0.0,1095.400024,736218000.0,0.0,271.600006,0.0,...,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0,2023-05-01 01:00:00


In [ ]:
if model == "gb":
    reg = GradientBoostingRegressor(n_estimators=1000, loss=loss)
elif model == "rf":
    reg = RandomForestRegressor(n_estimators=1000)

reg.fit(x_train.drop('pv_measurement', axis=1), x_train['pv_measurement'].fillna(0))

In [ ]:
pred = reg.predict(x_test.drop(['date_forecast_dt'], axis=1).fillna(0))

In [ ]:
x_test['prediction'] = pred
x_test['prediction'] = x_test['prediction'].clip(lower=0)
x_test[['date_forecast_dt', 'prediction']].set_index('date_forecast_dt').to_csv(f'pred_{location}_{model}_{version}_{loss}.csv')
x_test.head()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.scatter(x_test['date_forecast_dt'], x_test['prediction'] , c='blue', alpha=0.05, edgecolors='none')
#ax.set_yscale('log')
plt.show()